<div align="center">

# CMSC 320 Final Project: How Much Does Electricity Cost the US?

##### A Tutorial and Analysis by Jay Desmarais
***

</div>

<div align="center">

### Introduction

</div>

With a rising concern for the world's health and climate change, all things energy are a big topic. Researchers are finding the best ways to harness renewable energy efficiently, companies are turning away from the use of fossil fuels, people are buying electric cars, and electric people are starting to rely on their electric companies a little less, by installing their own solar panels. With all this happening, there are still large providers in every state that a huge portion of the United States rely on to get their electricity, and over the years, their means of obtaining and distributing said electricity may vary, whether they start relying more on nuclear plants, solar panels, or other more renewable sources, but how has this, among many other obstables, investments, failures, or more impacted the price of electricity. 

To dive deeper into understanding the pricing of electricity throughout the United States, will take a look at the U.S Department of Energy's data on the average price of electricity in each US state from the years 1990 - 2020 in their dataset titled "Average Price (Cents/kilowatthour) by State by Provider, 1990-2020". By taking a look at this data, we hope to find possible trends in prices accross the nation, in specific states or regions, and across time.

This analysis and tutorial will take a look at and walk through an example set of data and it's analysis to help guide the reader to be able to do their own analysis on a dataset of their choosing. This tutorial will focus on "The Data Science Pipeline", which is used as a process to collect data, manage it, explore results, and test against it. It consists of 5 main phases that we will touch on:

- Data Collection/curation + parsing
- Data Management
- Exploratory Data Analysis
- Hypothesis Testing
- Communication of insights attained

<div align="center">

### Required Tools and Libraries

</div>

The following tools and libraries are required for different aspects of this tutorial:

1. urllib.request 
2. pandas
3. numpys
4. matplotlib.pyplot
5. re
6. json
7. datetime
8. bs4

To import these packages, the following code can be used: 

In [51]:
import urllib.request as request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
from datetime import datetime
from bs4 import BeautifulSoup

<div align="center">

### Data Collection

</div>

This phase consists of finding a data set. This can be done by searching online and finding links to datasets, or finding downloads to save to your working directory. In the below example, I searched data.gov for an aggregate of datasets published by the Department of Energy. By doing so, I found a link to a json file that contains a json object of datasets collected and maintained by the Dept. of Energy. Once this json link was acquired, I used the urllib.request and json libraries to open and read in the json file to a json object. This takes the contents of that file and stores them in a variable named 'data' that I can later manipulate. Using this json object, I then went through all of the datasets until I found the one that I wanted to use, which I did by searching for the object with the title "Annual Electricity Price by State". I knew this was the title of the information I wanted beceause I looked throught the object manually to find the dataset I would like to access, but there were many more interesting datasets that can be used for further exploration. Once I found this specific dataset, I parsed through it to find the url of the excel spreadsheet that contained the data and used df.read_excel to read it in, making sure to specify which line the column headers are on. This stored the dataset in a pandas dataframe, which acts as a table with multiple rows and columns of the data that can be easily manipulated to further explore the data.

To read more about these specific functions or libraries and their uses, you can use the following resources:

- Department of Energy Open Data: https://dashboard.data.gov/offices/detail/49019/2021-12-31#metrics_total_records 
- urllib.request: https://docs.python.org/3/library/urllib.request.html
- json: https://docs.python.org/3/library/json.html
- pandas: https://pandas.pydata.org/docs/index.html
- pandas.read_excel: https://pandas.pydata.org/pandas-docs/version/0.20/generated/pandas.read_excel.html
- pandas.DataFrame: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

In [52]:

url = "https://www.energy.gov/sites/default/files/2022-05/PDL-05-04-2022.json"
response = request.urlopen(url)
data = json.loads(response.read())

for dataset in data['dataset']:
    if dataset['title'] == 'Annual Electricity Price by State':
        energy_dataset = dataset
        break

data_url = energy_dataset['distribution'][0]['downloadURL']

df = pd.read_excel(data_url, header=1)

df

,Year,State,Industry Sector Category,Residential,Commercial,Industrial,Transportation,Other,Total
0,2020,AK,Total Electric Industry,22.57,19.58,15.88,0.00,NaN,19.82
1,2020,AL,Total Electric Industry,12.58,11.55,5.87,0.00,NaN,9.84
2,2020,AR,Total Electric Industry,10.41,8.61,5.89,13.32,NaN,8.32
3,2020,AZ,Total Electric Industry,12.27,10.11,6.07,9.38,NaN,10.44
4,2020,CA,Total Electric Industry,20.45,17.53,14.27,10.07,NaN,18.00
...,...,...,...,...,...,...,...,...,...
4600,1990,WA,Full-Service Providers,4.39,4.15,2.39,NaN,3.13,3.40
4601,1990,WI,Full-Service Providers,6.63,5.78,3.99,NaN,6.47,5.37
4602,1990,WV,Full-Service Providers,5.90,5.36,3.56,NaN,8.19,4.73
4603,1990,WY,Full-Service Providers,5.97,5.17,3.47,NaN,7.90,4.21


Now that we have the data properly formatted in a pandas DataFrame, we can clearly see the format of this data. This data is collected each year in each of the states across the US and contains the avaerage residential, commercial, and industrial, transportation, and other costs of electricity by provider company. This results in a total of 4605 rows of data and 9 columns, labeled "Year", "State", "Industry Sector Category", "Residential", "Commercial", "Industrial", "Transportation", "Other", and "Total".

<div align="center">

### Data Management and Representation

</div>

After the data has been collected, we want to find ways to manage the data so it can be used properly later in the data analysis phase. Some things that you may want to consider as part of your data are the data types represented in each column, and any missing values. When you are managing your data, even when the data seems to be fully filled out, there may be some missing data. In our case, all years 2002 and prior have missing values in the 'Transporation' column, but values in the 'Other' column, but every year post 2002 has that switched, with values in the 'Transportation' column, but not in the 'Other' column. Because of this, we will be handling our missing data by setting it all to 0 and then combining the 'Transportation' and 'Other' columns into one. You can see this here:

In [53]:
df = df.fillna(0)
df['Other'] = df['Other'] + df['Transportation']
df = df.drop(['Transportation'], axis=1)

df

,Year,State,Industry Sector Category,Residential,Commercial,Industrial,Other,Total
0,2020,AK,Total Electric Industry,22.57,19.58,15.88,0.00,19.82
1,2020,AL,Total Electric Industry,12.58,11.55,5.87,0.00,9.84
2,2020,AR,Total Electric Industry,10.41,8.61,5.89,13.32,8.32
3,2020,AZ,Total Electric Industry,12.27,10.11,6.07,9.38,10.44
4,2020,CA,Total Electric Industry,20.45,17.53,14.27,10.07,18.00
...,...,...,...,...,...,...,...,...
4600,1990,WA,Full-Service Providers,4.39,4.15,2.39,3.13,3.40
4601,1990,WI,Full-Service Providers,6.63,5.78,3.99,6.47,5.37
4602,1990,WV,Full-Service Providers,5.90,5.36,3.56,8.19,4.73
4603,1990,WY,Full-Service Providers,5.97,5.17,3.47,7.90,4.21



The way you handle missing data can vary, but you always want to handle it in a way that makes sense with your data and how you are evaluating it. To learn more about missing data in pandas, visit https://www.geeksforgeeks.org/working-with-missing-data-in-pandas/.

These data types can be put into two main categories: Numerical data and Categorical data. Numerical data is data that uses integers or real numbers to express the data. This can be further broken down into interval scale or ratio scale.Interval scale represents data with a fixed but arbitrary interval, like dates, and degrees farenheit, where ratio scale is a scale of measurement that posesses a true-zero origin, like degrees kelvin or weight in grams. Categorical data on the other hand, is data that takes a value from a finite set, and can be further brokwn down into nominal data and ordinal data. Nominal data has values with names or categories, like in marital status or color, where ordinal data has a sort of ordering over the values, like in ratings ("strongly like", "like", "neutral", "strongly dislike").

Based on these types of data, there may be certain things that you want to do to manipulate your data and explore the relationships between the variables collected in the dataset. To demonstrate this, we will work to manage and represent our data in different ways by collecting different groupings, averages, and more, into different smaller, more focused DataFrames, or by creating new columns in the data tables, which can be seen here:

In [54]:
yearly_averages = df.groupby('Year').mean()

yearly_groupings = df.groupby('Year')

state_groupings = df.groupby('State')

company_groupings = df.groupby('Industry Sector Category')

yearly_averages

,Residential,Commercial,Industrial,Other,Total
Year,,,,,
1990,7.665385,6.968077,4.984038,7.384615,6.475577
1991,7.871923,7.123269,5.105385,7.546538,6.648654
1992,8.047500,7.248846,5.129423,7.725577,6.737500
1993,8.206923,7.351346,5.212308,7.917500,6.882308
1994,8.299423,7.337308,5.141154,7.999615,6.891731
1995,8.382500,7.342885,5.047500,8.132115,6.911923
1996,8.416038,7.393962,4.978868,8.176226,6.869245
1997,7.670000,7.145313,4.693437,6.688594,6.702969
1998,7.846563,6.900000,4.700156,6.581719,6.637812


Above this, we have created one new dataframe with averages of each columns by year

To learn more about the types of data, you can visit https://www.voxco.com/blog/categorical-data-vs-numerical-data/.

<div align="center">

### Exploratory Data Analysis

</div>

<div align="center">

### Hypothesis Testing

</div>

<div align="center">

### Communication of Insights Attained

</div>